In [1]:
import arelle
from arelle import Cntlr
from arelle.XbrlConst import parentChild
from collections import defaultdict
import pandas as pd
from pathlib import Path
controller = Cntlr.Cntlr()
model_manager = controller.modelManager


In [2]:
def load_taxonomy(taxonomy_path):
	model_xbrl = model_manager.load(taxonomy_path)
	return model_xbrl

def get_relationships(model_xbrl):
	rels = defaultdict(list)
	rs = model_xbrl.relationshipSet(parentChild)
	for rel in rs.modelRelationships:
		if rel.toModelObject.name not in rels[rel.fromModelObject.name]:
			rels[rel.fromModelObject.name].append(rel.toModelObject.name)
	return rels

def create_parent_child_table(rels):
	parent_child_table = {}
	for parent, children in rels.items():
		for child in children:
			parent_child_table[child] = parent
	df = pd.DataFrame.from_dict(parent_child_table, orient='index').reset_index()
	if df.empty:
		return df
	df.columns = ['child', 'parent']
	return df

## Schemas

These are the schemas that define the taxonomies


In [34]:
import pandas as pd
w = pd.read_excel("test.xslx", sheet_name=None)

In [108]:
model_xbrl = model_manager.load("test.xml")

In [111]:
model_xbrl.modelDocument[0]

TypeError: 'ModelDocument' object is not subscriptable

In [3]:
schemas = pd.read_csv("../data/02_intermediate/schemas.csv")

In [4]:
def parse_schema(schema):
	if 'org.uk' in schema:
		# return everything after .org.uk
		return schema.split(".org.uk")[1][1:]
	elif '.org/uk' in schema:
		return schema.split(".org/uk")[1][1:]
	else:
		return None
		

In [5]:
schemas['schema'].map(parse_schema)

0                         None
1                         None
2                         None
3         gaap/core/2009-09-01
4     reports/aurep/2009-09-01
5     reports/direp/2009-09-01
6              char/2016-01-01
7              char/2018-01-01
8              char/2019-01-01
9              char/2021-01-01
10             char/2022-01-01
11             char/2023-01-01
12              dpl/2023-01-01
13          fr/2014-09-01/core
14          fr/2018-01-01/core
15          fr/2019-01-01/core
16          fr/2021-01-01/core
17          fr/2022-01-01/core
18          fr/2023-01-01/core
19    reports/2014-09-01/aurep
20    reports/2014-09-01/direp
21    reports/2018-01-01/aurep
22    reports/2018-01-01/direp
23    reports/2019-01-01/aurep
24    reports/2019-01-01/direp
25    reports/2021-01-01/aurep
26    reports/2021-01-01/direp
27    reports/2022-01-01/aurep
28    reports/2022-01-01/direp
29    reports/2023-01-01/aurep
30    reports/2023-01-01/direp
31             secr/2019-04-01
32      

In [6]:
schema_paths = ["gaap/core/2009-09-01/uk-gaap-full-2009-09-01.xsd",
     "reports/aurep/2009-09-01/uk-aurep-full-2009-09-01.xsd",
     "reports/direp/2009-09-01/uk-direp-full-2009-09-01.xsd",
     "char/2016-01-01/char-2016-01-01.xsd",
	 "char/2018-01-01/char-2018-01-01.xsd",
     "char/2019-01-01/char-2019-01-01.xsd",
     "char/2021-01-01/char-2021-01-01.xsd",
     "char/2022-01-01/char-2022-01-01.xsd",
     "char/2023-01-01/char-2023-01-01.xsd",
     "dpl/2023-01-01/dpl-2023-01-01.xsd",
     "fr/2014-09-01/core/frc-core-full-2014-09-01.xsd",
     "fr/2018-01-01/core/frc-core-full-2018-01-01.xsd",
     "fr/2019-01-01/core/frc-core-full-2019-01-01.xsd",
     "fr/2021-01-01/core/frc-core-full-2021-01-01.xsd",
     "fr/2022-01-01/core/frc-core-full-2022-01-01.xsd",
     "fr/2023-01-01/core/frc-core-full-2023-01-01.xsd",
     "reports/2014-09-01/aurep/aurep-full-2014-09-01.xsd",
     "reports/2014-09-01/direp/direp-full-2014-09-01.xsd",
     "reports/2018-01-01/aurep/aurep-full-2018-01-01.xsd",
     "reports/2018-01-01/direp/direp-full-2018-01-01.xsd",
     "reports/2019-01-01/aurep/aurep-full-2019-01-01.xsd",
     "reports/2019-01-01/direp/direp-full-2019-01-01.xsd",
     "reports/2021-01-01/aurep/aurep-full-2021-01-01.xsd",
     "reports/2021-01-01/direp/direp-full-2021-01-01.xsd",
	 "reports/2022-01-01/aurep/aurep-full-2022-01-01.xsd",
    "reports/2022-01-01/direp/direp-full-2022-01-01.xsd",
     "reports/2023-01-01/aurep/aurep-full-2023-01-01.xsd",
     "reports/2023-01-01/direp/direp-full-2023-01-01.xsd",
     "secr/2019-04-01/secr.xsd"]
     
    

In [49]:
def get_concepts(model_xbrl):
	c = model_xbrl.nameConcepts
	return {concept[0].name: concept[0].label() for concept in c.values()}

In [7]:
def parse_taxonomy(path, path_dir):
	f = Path(f"../data/02_intermediate/schemas/{path_dir}")
	tax = load_taxonomy(str(f))
	concepts = tax.nameConcepts
	rels = get_relationships(tax)
	parent_child_table = create_parent_child_table(rels)
	parent_child_table['schema'] = path
	return parent_child_table

t = parse_taxonomy('fr/2018-01-01/core',schema_paths[0])

In [9]:
t = parse_taxonomy('gaap/core/2009-09-01',schema_paths[0])

In [13]:
taxon = load_taxonomy("../data/02_intermediate/schemas/gaap/core/2009-09-01/uk-gaap-full-2009-09-01.xsd")

In [27]:
from arelle.XbrlConst import conceptLabel, factExplanatoryFact

In [28]:
rs = taxon.relationshipSet(factExplanatoryFact)

In [80]:
def get_concepts(model_xbrl):
	c = model_xbrl.nameConcepts
	d = {}
	for concept in c.values():
		if len(concept) > 0:
			c1 = concept[0]
			l = c1.label()
			n = c1.name
			d[n] = l
			# d[n] = c1
	return d

In [81]:
c = get_concepts(taxon)

In [84]:
cdf = pd.Series(c).reset_index()                                                                                 

In [90]:
rels['CashBankInHand']

['CashInHand', 'CashBank']

In [86]:
rels = get_relationships(taxon)

In [87]:
rels

defaultdict(list,
            {'CapitalEmployed': ['CapitalReserves',
              'ShareholderFunds',
              'MinorityInterestBalanceSheet'],
             'FinancialInstrumentsPolicy': ['TradeOtherReceivablesPolicy',
              'TradeOtherPayablesPolicy',
              'CashCashEquivalentsPolicy',
              'BankBorrowingsPolicy',
              'ConvertibleLoanNotesPolicy',
              'EquityInstrumentsPolicy',
              'DerivativeFinancialInstrumentsPolicy',
              'DescriptionDesignationCriteriaForFinancialAssetsOrLiabilitiesFairValueThroughProfitOrLoss',
              'DescriptionCriteriaForDesignatingFinancialAssetsAsAvailable-for-sale',
              'PolicyOnUseAllowanceAccountForImpairedFinancialAssets',
              'DescriptionCriteriaForDeterminingNetGainsOrLossesOnFinancialInstruments',
              'DescriptionCriteriaForDeterminingThatAnImpairmentLossHasOccurred',
              'PolicyOnFinancialAssetsThatAreSubjectToRenegotiation'],
      

In [95]:
t[t['child'].str.contains('ShareNetAssets')]

,child,parent,schema
1500,ShareNetAssetsLiabilitiesJoint-venturesAssocia...,SegmentalAnalysisAssetsLiabilitiesHeading,gaap/core/2009-09-01
1525,ShareNetAssetsLiabilitiesJoint-venturesAssocia...,ShareNetAssetsLiabilitiesJoint-venturesAssocia...,gaap/core/2009-09-01


In [96]:
t = pd.read_html("http://localhost:8080/rest/xbrl/notebooks;test.xml/factTable")

In [98]:
t = pd.read_json("http://localhost:8080/rest/xbrl/view?file=notebooks/test.xml&view=factTable&media=json")

In [101]:
import requests
r = requests.get("http://localhost:8080/rest/xbrl/view?file=notebooks/test.xml&view=factTable&media=json").json()

In [107]:
r['factTable'][1]

['linkRole',
 {'role': 'http://www.xbrl-uk.org/ExtendedLinkRoles/AE-Directors-Report',
  'definition': "AE Directors' Report"},
 {},
 ['facts',
  {'concept': 'uk-gaap-ae:DirectorsReport'},
  {},
  ['facts', {'concept': 'uk-gcd:EntityBusinessDescription'}, {}],
  ['facts', {'concept': 'uk-gaap-ae:DateApprovalDirectorsReport'}, {}],
  ['facts', {'concept': 'uk-gaap-ae:DirectorsReportGeneralText'}, {}],
  ['facts', {'concept': 'uk-gaap-ae:PoliticalCharitableDonationsText'}, {}],
  ['facts', {'concept': 'uk-gaap-ae:CompanyDormant'}, {}],
  ['facts', {'concept': 'uk-gaap-ae:CompanyNotDormant'}, {'y2014': 'true'}],
  ['facts', {'concept': 'uk-gaap-ae:CompanyNon-trading'}, {}],
  ['facts',
   {'concept': 'uk-gaap-ae:ListDirectorsExecutives'},
   {},
   ['facts', {'concept': 'uk-gaap-pt:DirectorOrExecutivesName'}, {}],
   ['facts', {'concept': 'uk-gaap-pt:DateAssumedPosition'}, {}],
   ['facts', {'concept': 'uk-gaap-ae:DateResignation'}, {}]],
  ['facts', {'concept': 'uk-gaap-ae:CompanySecreta

In [75]:
taxons = []
for i,s in enumerate(schemas['schema'].map(parse_schema).dropna().to_list()):
	if s is not None:
		print(s)
		try:
			t = parse_taxonomy(s, schema_paths[i])
			taxons.append(t)
		except Exception as e:
			print(f"Failed to parse {s}")
		

gaap/core/2009-09-01
reports/aurep/2009-09-01
reports/direp/2009-09-01
char/2016-01-01
Failed to parse char/2016-01-01
char/2018-01-01
Failed to parse char/2018-01-01
char/2019-01-01
Failed to parse char/2019-01-01
char/2021-01-01
Failed to parse char/2021-01-01
char/2022-01-01
Failed to parse char/2022-01-01
char/2023-01-01
Failed to parse char/2023-01-01
dpl/2023-01-01
fr/2014-09-01/core
fr/2018-01-01/core
fr/2019-01-01/core
fr/2021-01-01/core
fr/2022-01-01/core
fr/2023-01-01/core
reports/2014-09-01/aurep
reports/2014-09-01/direp
reports/2018-01-01/aurep
reports/2018-01-01/direp
reports/2019-01-01/aurep
reports/2019-01-01/direp
reports/2021-01-01/aurep
reports/2021-01-01/direp
reports/2022-01-01/aurep
reports/2022-01-01/direp
reports/2023-01-01/aurep
reports/2023-01-01/direp
secr/2019-04-01


In [59]:
parse_taxonomy('secr/2019-04-01',schema_paths[-1])

,child,parent,schema
0,StartDateForPeriodCoveredBySECRReport,StreamlinedEnergyCarbonReportingHeading,secr/2019-04-01
1,EndDateForPeriodCoveredBySECRReport,StreamlinedEnergyCarbonReportingHeading,secr/2019-04-01
2,DescriptionAvailabilityEmissionsInformationAny...,StreamlinedEnergyCarbonReportingHeading,secr/2019-04-01
3,DescriptionMethodologiesUsedToCalculateData,StreamlinedEnergyCarbonReportingHeading,secr/2019-04-01
4,DescriptionAssessmentMarket-basedAdditionality,StreamlinedEnergyCarbonReportingHeading,secr/2019-04-01
...,...,...,...
63,EmissionsIndirectCooling,EmissionsIndirectTotal,secr/2019-04-01
64,DisclosureGovernance,DisclosuresTaskForceOnClimate-relatedFinancial...,secr/2019-04-01
65,DisclosureStrategy,DisclosuresTaskForceOnClimate-relatedFinancial...,secr/2019-04-01
66,DisclosureRiskManagement,DisclosuresTaskForceOnClimate-relatedFinancial...,secr/2019-04-01


In [76]:
taxon_df = pd.concat(taxons)

In [84]:
taxon_df.query("child == 'StaffCostsEmployeeBenefitsExpense'")

,child,parent,schema,root
139,StaffCostsEmployeeBenefitsExpense,StaffEmploymentRelatedCostsHeading,dpl/2023-01-01,dpl
579,StaffCostsEmployeeBenefitsExpense,AdministrativeExpenses,fr/2014-09-01/core,fr
605,StaffCostsEmployeeBenefitsExpense,AdministrativeExpenses,fr/2018-01-01/core,fr
644,StaffCostsEmployeeBenefitsExpense,AdministrativeExpenses,fr/2019-01-01/core,fr
690,StaffCostsEmployeeBenefitsExpense,PayRatioInformationHeading,fr/2021-01-01/core,fr
374,StaffCostsEmployeeBenefitsExpense,PayRatioInformationHeading,fr/2022-01-01/core,fr
395,StaffCostsEmployeeBenefitsExpense,PayRatioInformationHeading,fr/2023-01-01/core,fr
83,StaffCostsEmployeeBenefitsExpense,PayRatioInformationHeading,reports/2021-01-01/direp,reports
139,StaffCostsEmployeeBenefitsExpense,StaffEmploymentRelatedCostsHeading,reports/2022-01-01/aurep,reports
153,StaffCostsEmployeeBenefitsExpense,PayRatioInformationHeading,reports/2022-01-01/direp,reports


In [82]:
taxon_df['root'] = taxon_df['schema'].str.split("/").str[0]

In [79]:
taxon_df.to_csv("../data/02_intermediate/taxonomy.csv", index=False)

In [83]:
taxon_df.groupby(['child','root'])['parent'].nunique().sort_values(ascending=False)

child                                                                            root   
NotApplicableDefault                                                             fr         5
PoliticalDonations                                                               reports    4
DescriptionPrincipalActivities                                                   reports    3
DescriptionArrangementsRelatedToEmployeesSuchAsProvisionInformationConsultation  reports    3
DateSigningDirectorsReport                                                       reports    3
                                                                                           ..
ExplorationEvaluationCostsMeasurementPolicy                                      fr         1
ExplorationEvaluationCostsSwapsPolicy                                            fr         1
ExplorationEvaluationExpenditurePolicyExtractiveIndustries                       fr         1
ExplorationLicenses                                              

In [13]:
schema_paths = schemas['schema'].map(parse_schema).dropna().to_list()

In [35]:
s = ["gaap/core/2009-09-01/uk-gaap-full-2009-09-01.xsd",
     "reports/aurep/2009-09-01/uk-aurep-full-2009-09-01.xsd",
     "reports/direp/2009-09-01/uk-direp-full-2009-09-01.xsd",
     "char/2016-01-01/char-2016-01-01.xsd",
	 "char/2018-01-01/char-2018-01-01.xsd",
     "char/2019-01-01/char-2019-01-01.xsd",
     "char/2021-01-01/char-2021-01-01.xsd",
     "char/2022-01-01/char-2022-01-01.xsd",
     "char/2023-01-01/char-2023-01-01.xsd",
     "dpl/2023-01-01/dpl-2023-01-01.xsd",
     "fr/core/frc-core-full-2014-09-01.xsd",
     "fr/core/frc-core-full-2018-01-01.xsd",
     "fr/core/frc-core-full-2019-01-01.xsd",
     "fr/core/frc-core-full-2021-01-01.xsd",
     "fr/core/frc-core-full-2022-01-01.xsd",
     "fr/core/frc-core-full-2023-01-01.xsd",
     "reports/2014-09-01/aurep/aurep-full-2014-09-01.xsd",
     "reports/2014-09-01/direp/direp-full-2014-09-01.xsd",
     "reports/2018-01-01/aurep/aurep-full-2018-01-01.xsd",
     "reports/2018-01-01/direp/direp-full-2018-01-01.xsd",
     "reports/2019-01-01/aurep/aurep-full-2019-01-01.xsd",
     "reports/2019-01-01/direp/direp-full-2019-01-01.xsd",
     "reports/2021-01-01/aurep/aurep-full-2021-01-01.xsd",
     "reports/2021-01-01/direp/direp-full-2021-01-01.xsd",
	 "reports/2022-01-01/aurep/aurep-full-2022-01-01.xsd",
     "reports/2023-01-01/aurep/aurep-full-2023-01-01.xsd",
     "reports/2023-01-01/direp/direp-full-2023-01-01.xsd",
     "secr/2019-04-01/secr.xsd"]
     